In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

In [2]:
file = "sqlite_db_pythonsqlite.db"
db = sql.connect(file)

In [3]:
queryf = "SELECT * FROM Facilities"
querym = "SELECT * FROM Members"
queryb = "SELECT * FROM Bookings"

In [4]:
facilities = pd.read_sql_query(queryf, db)
members = pd.read_sql_query(querym, db)
bookings = pd.read_sql_query(queryb, db)

In [5]:
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [6]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [7]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [8]:
bookings['starttime'] = pd.to_datetime(bookings['starttime'])

In [9]:
bookings['booking_month'] = pd.DatetimeIndex(bookings['starttime']).month

In [10]:
bookingsXmembers = pd.merge(bookings, members, on='memid', how='left')

In [11]:
ccfull = pd.merge(bookingsXmembers, facilities, on='facid', how='left')

In [12]:
ccfull.head()

,bookid,facid,memid,starttime,slots,booking_month,surname,firstname,address,zipcode,telephone,recommendedby,joindate,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,3,1,2012-07-03 11:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Table Tennis,0.0,5.0,320,10
1,1,4,1,2012-07-03 08:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Massage Room 1,9.9,80.0,4000,3000
2,2,6,0,2012-07-03 18:00:00,2,7,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00,Squash Court,3.5,17.5,5000,80
3,3,7,1,2012-07-03 19:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Snooker Table,0.0,5.0,450,15
4,4,8,1,2012-07-03 10:00:00,1,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Pool Table,0.0,5.0,400,15


In [13]:
def f(row):
        if row['memid'] != 0:
            value = row['slots'] * row['membercost']
        elif row['memid'] == 0:
            value = row['slots'] * row['guestcost']
        else:
            value = 0
        return value

In [14]:
ccfull['cost'] = ccfull.apply(f, axis = 1)

In [15]:
ccfull.head()

,bookid,facid,memid,starttime,slots,booking_month,surname,firstname,address,zipcode,telephone,recommendedby,joindate,name,membercost,guestcost,initialoutlay,monthlymaintenance,cost
0,0,3,1,2012-07-03 11:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Table Tennis,0.0,5.0,320,10,0.0
1,1,4,1,2012-07-03 08:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Massage Room 1,9.9,80.0,4000,3000,19.8
2,2,6,0,2012-07-03 18:00:00,2,7,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00,Squash Court,3.5,17.5,5000,80,35.0
3,3,7,1,2012-07-03 19:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Snooker Table,0.0,5.0,450,15,0.0
4,4,8,1,2012-07-03 10:00:00,1,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Pool Table,0.0,5.0,400,15,0.0


In [16]:
q10 = ccfull.groupby('name')['cost'].sum()

In [17]:
q10 = pd.DataFrame(q10)

In [18]:
q10 = q10[q10['cost'] < 1000]

In [19]:
q10['cost'].sort_values(ascending=False)

name
Pool Table       270.0
Snooker Table    240.0
Table Tennis     180.0
Name: cost, dtype: float64

In [20]:
# Answer to Q10

q10

,cost
name,
Pool Table,270.0
Snooker Table,240.0
Table Tennis,180.0


In [21]:
members['lastfirst'] = members['surname'] + ' ' + members['firstname']

In [22]:
id_name = dict(zip(members['memid'],members['lastfirst']))

In [23]:
q11 = members

In [24]:
q11['recommendedby'] = pd.to_numeric(q11['recommendedby'],errors='ignore')

In [25]:
q11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   memid          31 non-null     int64  
 1   surname        31 non-null     object 
 2   firstname      31 non-null     object 
 3   address        31 non-null     object 
 4   zipcode        31 non-null     int64  
 5   telephone      31 non-null     object 
 6   recommendedby  22 non-null     float64
 7   joindate       31 non-null     object 
 8   lastfirst      31 non-null     object 
dtypes: float64(1), int64(2), object(6)
memory usage: 2.3+ KB


In [26]:
q11['recommended_name'] = q11['recommendedby'].map(id_name)

In [27]:
q11

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate,lastfirst,recommended_name
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,NaN,2012-07-01 00:00:00,GUEST GUEST,NaN
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,NaN,2012-07-02 12:02:05,Smith Darren,NaN
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,NaN,2012-07-02 12:08:23,Smith Tracy,NaN
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,NaN,2012-07-03 09:32:15,Rownam Tim,NaN
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1.0,2012-07-03 10:25:05,Joplette Janice,Smith Darren
5,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1.0,2012-07-09 10:44:09,Butters Gerald,Smith Darren
6,6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,NaN,2012-07-15 08:52:55,Tracy Burton,NaN
7,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4.0,2012-07-25 08:59:12,Dare Nancy,Joplette Janice
8,8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3.0,2012-07-25 16:02:35,Boothe Tim,Rownam Tim
9,9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6.0,2012-07-25 17:09:05,Stibbons Ponder,Tracy Burton


In [28]:
q11 = q11[['memid','surname','firstname','recommended_name']]

In [29]:
q11 = q11.sort_values(by='surname')

In [30]:
q11 = q11.fillna(value='No recommendation given')

In [31]:
# Answer to Q11

q11

,memid,surname,firstname,recommended_name
15,15,Bader,Florence,Stibbons Ponder
16,16,Baker,Timothy,Farrell Jemima
12,12,Baker,Anne,Stibbons Ponder
8,8,Boothe,Tim,Rownam Tim
5,5,Butters,Gerald,Smith Darren
20,22,Coplin,Joan,Baker Timothy
29,36,Crumpet,Erica,Smith Tracy
7,7,Dare,Nancy,Joplette Janice
24,28,Farrell,David,No recommendation given
13,13,Farrell,Jemima,No recommendation given


In [32]:
q12_q13 = ccfull

In [33]:
q12_q13 = q12_q13[q12_q13.memid != 0]

In [34]:
q12_q13

,bookid,facid,memid,starttime,slots,booking_month,surname,firstname,address,zipcode,telephone,recommendedby,joindate,name,membercost,guestcost,initialoutlay,monthlymaintenance,cost
0,0,3,1,2012-07-03 11:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Table Tennis,0.0,5.0,320,10,0.0
1,1,4,1,2012-07-03 08:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Massage Room 1,9.9,80.0,4000,3000,19.8
3,3,7,1,2012-07-03 19:00:00,2,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Snooker Table,0.0,5.0,450,15,0.0
4,4,8,1,2012-07-03 10:00:00,1,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Pool Table,0.0,5.0,400,15,0.0
5,5,8,1,2012-07-03 15:00:00,1,7,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,Pool Table,0.0,5.0,400,15,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,4038,8,29,2012-09-30 16:30:00,2,9,Worthington-Smyth,Henry,"55 Jagbi Way, North Reading",97676,(855) 894-3758,2,2012-09-17 12:27:15,Pool Table,0.0,5.0,400,15,0.0
4039,4039,8,29,2012-09-30 18:00:00,1,9,Worthington-Smyth,Henry,"55 Jagbi Way, North Reading",97676,(855) 894-3758,2,2012-09-17 12:27:15,Pool Table,0.0,5.0,400,15,0.0
4040,4040,8,21,2012-09-30 18:30:00,1,9,Mackenzie,Anna,"64 Perkington Lane, Reading",64577,(822) 661-2898,1,2012-08-26 09:32:05,Pool Table,0.0,5.0,400,15,0.0
4041,4041,8,16,2012-09-30 19:00:00,1,9,Baker,Timothy,"329 James Street, Reading",58393,833-941-0824,13,2012-08-15 10:34:25,Pool Table,0.0,5.0,400,15,0.0


In [35]:
q12 = q12_q13.pivot_table(values='slots',index='memid',columns='name',aggfunc='sum')

In [36]:
q12 = q12.fillna(0)

In [37]:
q12['Member'] = q11['memid'].map(id_name)

In [38]:
q12cols = q12.columns.tolist()

In [39]:
q12cols = q12cols[-1:] + q12cols[:-1]

In [40]:
q12 = q12[q12cols]

In [41]:
# Answer to Q12, number of slots booked per Member

q12

name,Member,Badminton Court,Massage Room 1,Massage Room 2,Pool Table,Snooker Table,Squash Court,Table Tennis,Tennis Court 1,Tennis Court 2
memid,,,,,,,,,,
1,Smith Darren,432.0,58.0,0.0,28.0,24.0,30.0,56.0,0.0,57.0
2,Smith Tracy,102.0,12.0,0.0,64.0,90.0,12.0,56.0,93.0,6.0
3,Rownam Tim,12.0,176.0,4.0,282.0,0.0,0.0,150.0,18.0,18.0
4,Joplette Janice,0.0,24.0,4.0,27.0,142.0,30.0,18.0,57.0,24.0
5,Butters Gerald,63.0,66.0,2.0,6.0,72.0,18.0,2.0,171.0,9.0
6,Tracy Burton,6.0,62.0,0.0,30.0,40.0,78.0,48.0,93.0,9.0
7,Dare Nancy,30.0,38.0,10.0,19.0,46.0,0.0,10.0,81.0,33.0
8,Boothe Tim,36.0,76.0,0.0,26.0,90.0,24.0,8.0,12.0,168.0
9,Stibbons Ponder,48.0,40.0,0.0,12.0,40.0,4.0,6.0,3.0,96.0


In [44]:
q13 = q12_q13.pivot_table(values='slots',index='booking_month',columns='name',aggfunc='sum')

In [45]:
# Answer to Q13, slots booked per month per facility

q13

name,Badminton Court,Massage Room 1,Massage Room 2,Pool Table,Snooker Table,Squash Court,Table Tennis,Tennis Court 1,Tennis Court 2
booking_month,,,,,,,,,
7,165,166,8,110,140,50,98,201,123
8,414,316,18,303,316,184,296,339,345
9,507,402,28,443,404,184,400,417,414
